# MindSpore——基于多模型的狗品种分类实现

本文使用[MindSpore Vision](https://mindspore.cn/vision/docs/zh-CN/r0.1/index.html)中的多种网络模型来开发一个AI应用（狗的品种分类），通过比较不同模型的分类验证精度并将训练效果最佳的网络模型部署到Android手机上，执行推理和部署功能。

## 训练模型

### .Mobilenetv2 、resnet50、resnet101

## 数据准备与加载

### 下载数据集

首先需要下载本案例所用到的[狗的品种分类数据集](https://cloud.tsinghua.edu.cn/f/80013ef29c5f42728fc8/?dl=1)，该数据集中狗的种类有130种,图片数量为70482。

![狗的分类识别数据集图.png](./image/狗的分类识别数据集图.png)

数据集的目录结构如下所示：

```text
datasets
└── low-resolution
    ├── Airedale
    │   ├── n10001.jpeg
    │   └── n10002.jpeg
    │   └── ...
    │    ...
    ├── basenji
    │   ├── n12701.jpg
    │   └── n12702.jpg
    │   └── ...
    │    ...
```

## 数据集预处理

通过上面的链接，在下载数据集后，为了使模型产生更好的效果，这里我们对数据集进行了预处理，主要包括：图片聚焦、图片裁剪以及数据集的划分。

### 一、图片聚焦

![yolo.png](./image/yolo.png)

#### 技术实现：调用YOLOv5接口(基于以前项目中部署的YOLO(torch)，只用于做数据预处理，后续各种模型训练与测试均为mindspore实现)

[src文件下载以及具体环境配置参考](https://github.com/ultralytics/yolov5)

#### 采用yolov5s.pt模型，其识别类型为80种。经过YOLO接口处理后，我们就可以得到原数据集图片被识别为各对象的位置坐标。

#### 下方为YOLO接口调用的代码。此过程是在本机上完成，所以用到的是绝对路径。

#### 如有需要可以根据上方链接下载数据集切换路径就可以在你的本机上完成。

In [ ]:
import cv2
import src.detect as detect
import os
from pathlib import Path
from PIL import Image

images_path = 'D:\\download\\low-resolution\\low-resolution'#原数据集path
path_list = os.listdir(images_path)
for i in range(len(path_list)):
    images_path1 = 'D:\\download\\low-resolution\\low-resolution' + "\\" + path_list[i]
    detect_api = detect.DetectAPI(exist_ok=True, source=images_path1, weights='yolov5s.pt', name='exp_dog_' + i.__str__())
    label = detect_api.run()#接口调用

### 二、图片裁剪

#### 技术实现：由上一步我们就得到了原数据集图片被识别为各个对象的位置坐标，如下表所示：

| id | x | y | w | h | confidence |
| :----:| :----: | :----: | :----:| :----: | :----: |
| 16 | 0.522917 | 0.606944 | 0.320833 | 0.719444 | 0.870236 |

id:识别图片的类别

x:识别框中心坐标中横坐标占图片长度的比例

y:识别框中心坐标中纵坐标坐标占图片长度的比例

w:识别框长度占图片长度的比例

h:识别框宽度占图片长度的比例

confidence:识别对象置信度

利用以上坐标通过下方的代码进行裁剪来获得识别对象的图像。

In [ ]:
id_set = set() # 存放识别的类别id
path_label = "./runs/detect/exp_dog/labels" # yolo识别图像后的标签
path_image = images_path # 原图像
path_list_label = os.listdir(path_label)
path_list_image = os.listdir(path_image)
#path_list_label.sort(key=lambda x:int(x.split('.')[1]))
#path_list_image.sort(key=lambda x:int(x.split('.')[1]))
for i in range(len(path_list_label)):
    path1 = path_image + "\\" + path_list_image[i]
    path2 = path_label + "\\" + path_list_label[i]
    img = cv2.imread(path1)
    box = Path(path2).read_text()
    box = box.split("\n")
    for j in range(len(box)-1):
        dh, dw, _ = img.shape
        class_id, x_center, y_center, w, h, conf = box[j].strip().split()
        id_set.add(int(class_id))
        x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
        x_center = round(x_center * dw)
        y_center = round(y_center * dh)
        w = round(w * dw)
        h = round(h * dh)
        x = round(x_center - w / 2)
        y = round(y_center - h / 2)
        imgCrop = img[y:y + h, x:x + w]
        conf = float(conf)
        path3 = "data/class_dog/" + class_id + "/" + j.__str__() + "." + path_list_image[i]  # 保存类别图像的位置
        cv2.imwrite(path3, imgCrop)
        path4 = "data/class_label_dog/" + class_id + "/" + j.__str__() + "." + path_list_image[i]  # 保存类别图像位于原图片坐标的位置
        file = open(path4 + ".txt", "w")
        file.seek(0)
        file.truncate()
        file.write(x.__str__() + "," + y.__str__() + "," + (x + w).__str__() + "," + (h + y).__str__())
        file.close()

### 三、数据集的划分

在将数据集中的狗裁剪出来后，按照原数据集的目录结构，将图片重新组成数据集。现在的数据集目录结构与代码中的数据集目录结构不相同，对数据集进行处理，将其转换为代码中的数据集目录结构。

处理后的数据集的目录结构如下所示（将每一类按照train：val：infer = 8：1：1划分）：

```text
datasets
├── infer
    │   ├── Airedale
    │   │   ├── n10001.jpeg
    │   │   └── n10002.jpeg
    │   │   └── ...
    │   │   ...
    │   ├── basenji
    │   │   ├── n12701.jpg
    │   │   └── n12702.jpg
    │   │   └── ...
    │   │   ...
    │  
    ├── train
    │   ├── Airedale
    │   │   └── ...
    │   │   ...
    │   ├── basenji
    │   │   └── ...
    │   │   ...
    │  
    └── val
        ├── Airedale
        │   └── ...
        │   ...
        ├── basenji
        │   └── ...
        │   ...

```

以下为数据集划分的代码：

In [ ]:
#代码
import os
import shutil
import tqdm

# list_name用来存储130种狗的种类名
list_name = []
# 数据集的存储路径
load_path = 'D:/download/low-resolution/low-resolution'
names = os.listdir(load_path)
names = names[0:-1]
# 路径中的文件名称为  x-xx-xxx，xxx就是狗的种类名
names.sort(key=lambda x: int(x.split('-')[0]))
for name in names:
    list_name.append(name.split('-')[2])

# 数据集的存储路径
s1 = 'D:/download/low-resolution/low-resolution'
# 划分后数据集的存储路径
s2 = 'D:/download/low-resolution/datasets2/'
# train、val和infer的文件路径
train = s2 + 'train'
val = s2 + 'val'
infer = s2 + 'infer'
# 创建文件目录
os.makedirs(train)
os.makedirs(val)
os.makedirs(infer)
for i in range(130):
    os.makedirs(train + '/' + list_name[i])
    os.makedirs(val + '/' + list_name[i])
    os.makedirs(infer + '/' + list_name[i])
    Files = os.listdir(s1)
    path = s1 + '/' + Files[i]
    files = os.listdir(path)
    for j in tqdm.tqdm(range(len(files))):
        # 按照8：1：1划分
        old_file_path = path + '/' + files[j]
        if j < len(files) * 0.8:
            new_file_path = train + '/' + list_name[i] + '/' + files[j]
        elif j < len(files) * 0.9:
            new_file_path = val + '/' + list_name[i] + '/' + files[j]
        else:
            new_file_path = infer + '/' + list_name[i] + '/' + files[j]
        # 从指定文件复制图片到另一指定文件
        shutil.copy(old_file_path, new_file_path)

经过处理后的数据集的保存路径如下所示：

```text
──训练集：./datasets/train

──验证集：./datasets/val

──推理集:../datasets/infer

```

## 加载数据集

定义 `listdir`函数获取狗的种类名索引。

In [2]:
import os
def listdir(path, list_name):
    i = 0
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            list_name[file] = i
            i += 1

定义 `create_dataset`函数加载狗和牛角包数据集，对数据集进行图像增强操作并设置数据集batch_size大小。

In [3]:
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as transforms

def create_dataset(path, batch_size=20, train=True, image_size=224):

    list_name = {}
    train_path = "/home/user2/LiDexin/datasets/train"
    listdir(train_path, list_name)
    print(list_name)
    dataset = ds.ImageFolderDataset(path, num_parallel_workers=8, class_indexing=list_name)

    # 图像增强操作
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]
    if train:
        trans = [
            transforms.RandomCropDecodeResize(image_size, scale=(0.08, 1.0), ratio=(0.75, 1.333)),
            transforms.RandomHorizontalFlip(prob=0.5),
            transforms.Normalize(mean=mean, std=std),
            transforms.HWC2CHW()
        ]
    else:
        trans = [
            transforms.Decode(),
            transforms.Resize(256),
            transforms.CenterCrop(image_size),
            transforms.Normalize(mean=mean, std=std),
            transforms.HWC2CHW()
        ]

    dataset = dataset.map(operations=trans, input_columns="image", num_parallel_workers=8)
    # 设置batch_size的大小，若最后一次抓取的样本数小于batch_size，则丢弃
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

加载训练数据集和验证数据集用于后续的模型训练和验证。

In [4]:
# 加载训练数据集
train_path = "/home/user2/LiDexin/datasets/train"
dataset_train = create_dataset(train_path, train=True)

# 加载验证数据集
val_path = "/home/user2/LiDexin/datasets/val"
dataset_val = create_dataset(val_path, train=False)

{'affenpinscher': 0, 'Afghan_hound': 1, 'African_hunting_dog': 2, 'Airedale': 3, 'American_Staffordshire_terrier': 4, 'Appenzeller': 5, 'Australian_Shepherd': 6, 'Australian_terrier': 7, 'basenji': 8, 'basset': 9, 'beagle': 10, 'Bedlington_terrier': 11, 'Bernese_mountain_dog': 12, 'Bichon_Frise': 13, 'black_and_tan_coonhound': 14, 'Black_sable': 15, 'Blenheim_spaniel': 16, 'bloodhound': 17, 'bluetick': 18, 'Border_collie': 19, 'Border_terrier': 20, 'borzoi': 21, 'Boston_bull': 22, 'Bouvier_des_Flandres': 23, 'boxer': 24, 'Brabancon_griffo': 25, 'briard': 26, 'Brittany_spaniel': 27, 'bull_mastiff': 28, 'cairn': 29, 'Cane_Carso': 30, 'Cardigan': 31, 'Chesapeake_Bay_retriever': 32, 'Chihuahua': 33, 'Chinese_Crested_Dog': 34, 'chinese_rural_dog': 35, 'chow': 36, 'clumber': 37, 'cocker_spaniel': 38, 'collie': 39, 'curly_coated_retriever': 40, 'Dandie_Dinmont': 41, 'dhole': 42, 'dingo': 43, 'Doberman': 44, 'English_foxhound': 45, 'English_setter': 46, 'English_springer': 47, 'EntleBucher': 4

## 模型训练

本案例将采用MobileNetV2,ResNet50,ResNet101三个模型对数据集进行训练或微调。通过测试集测试后，我们从三个模型中选取效果最好的模型部署到手机。

### MobileNet V2模型原理

MobileNet网络是由Google团队于2017年提出的专注于移动端、嵌入式或IoT设备的轻量级CNN网络，相比于传统的卷积神经网络，MobileNet网络使用深度可分离卷积（Depthwise Separable Convolution）的思想在准确率小幅度降低的前提下，大大减小了模型参数与运算量。并引入宽度系数 $\alpha$ 和分辨率系数 $\beta$ 使模型满足不同应用场景的需求。

由于MobileNet网络中Relu激活函数处理低维特征信息时会存在大量的丢失，所以MobileNetV2网络提出使用倒残差结构（Inverted residual block）和Linear Bottlenecks来设计网络，以提高模型的准确率，且优化后的模型更小。

![mobilenet](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/tutorials/source_zh_cn/beginner/images/mobilenet.png)

图中Inverted residual block结构是先使用1x1卷积进行升维，然后使用3x3的DepthWise卷积，最后使用1x1的卷积进行降维，与Residual block结构相反。Residual block是先使用1x1的卷积进行降维，然后使用3x3的卷积，最后使用1x1的卷积进行升维。

> [详细内容可参见MobileNet V2论文](https://arxiv.org/pdf/1801.04381.pdf)

### ResNet50模型原理

ResNet在Pytorch官方代码中有5种不同深度的结构，分别为18、34、50、101、152（各网络深度指的是“需要通过训练更新参数“的层数，如卷积层，全连接层等），和论文完全一致。

根据Block类型，可以将这五种ResNet分为两类：
1）基于BasicBlock，浅层网络18，34都是由BasicBlock 搭成；
2）基于Bottleneck，深层网络50，101，152是由Bottlen搭建而成；
Block相当于积木，每个layer都由Block搭建构成，再由layer组成整个网络。每中ResNet都是4个layer（不算一开始的7*7卷积层和maxpooling层）。

![resnet结构表.png](./image/resnet结构表.png)

Resnet50 网络中包含了 49 个卷积层、1个全连接层。如下图所示，Resnet50网络结构可以分成七个部分，第一部分不包含残差块，主要对输入进行卷积、正则化、激活函数、最大池化的计算。第二、三、四、五部分结构都包含了残差块，图中的绿色图块不会改变残差块的尺寸，只用于改变残差块的维度。在 Resnet50 网 络 结 构 中 ， 残 差 块 都 有 三 层 卷 积 ， 那 网 络 总 共 有1+3×（3+4+6+3）=49个卷积层，加上最后的全连接层总共是 50 层，这也是Resnet50 名称的由来。网络的输入为 224×224×3，经过前五部分的卷积计算，输出为 7×7×2048，池化层会将其转化成一个特征向量，最后分类器会对这个特征向量进行计算并输出类别概率。

![resnet50结构图.png](./image/resnet50结构图.png)

### ResNet101模型原理

上表是Resnet不同的结构，上表一共提出了5中深度的ResNet，分别是18，34，50，101和152，首先看表的最左侧，我们发现所有的网络都分成5部分，分别是：conv1，conv2_x，conv3_x，conv4_x，conv5_x，之后的其他论文也会专门用这个称呼指代ResNet50或者101的每部分。 例如：101-layer那列，101-layer指的是101层网络，首先有个输入7x7x64的卷积，然后经过3 + 4 + 23 + 3 = 33个building block，每个block为3层，所以有33 x 3 = 99层，最后有个fc层(用于分类)，所以1 + 99 + 1 = 101层，确实有101层网络； 注：101层网络仅仅指卷积或者全连接层，而激活层或者Pooling层并没有计算在内；我们关注50-layer和101-layer这两列，可以发现，它们唯一的不同在于conv4_x，ResNet50有6个block，而ResNet101有23个block，两者之间差了17个block，也就是17 x 3 = 51层。

### 下载预训练模型

#### MobileNet V2模型

下载案例所需的[MobileNetV2预训练模型的ckpt文件](https://download.mindspore.cn/vision/classification/mobilenet_v2_1.0_224.ckpt)，预训练模型的宽度系数$\alpha= 1.0$，输入图像大小为(224, 224), 将下载的预训练模型保存在当前目录下。使用MindSpore Vision中的`DownLoad`下载预训练模型文件到当前目录下，示例代码如下所示：

In [4]:
from mindvision.dataset import DownLoad

models_url1 = "https://download.mindspore.cn/vision/classification/mobilenet_v2_1.0_224.ckpt"

dl1 = DownLoad()
# 下载预训练模型文件
dl1.download_url(models_url1)

#### ResNet50模型

下载案例所需的[ResNet50预训练模型的ckpt文件](https://download.mindspore.cn/vision/classification/mobilenet_v2_1.0_224.ckpt)，输入图像大小为(224, 224), 将下载的预训练模型保存在当前目录下。使用MindSpore Vision中的`DownLoad`下载预训练模型文件到当前目录下，示例代码如下所示：

In [ ]:
from mindvision.dataset import DownLoad

models_url = "https://download.mindspore.cn/vision/classification/resnet50_224.ckpt"

dl = DownLoad()
# 下载预训练模型文件
dl.download_url(models_url)

#### ResNet101模型

下载案例所需的[ResNet101预训练模型的ckpt文件](https://download.mindspore.cn/vision/classification/resnet101_224.ckpt)，输入图像大小为(224, 224), 将下载的预训练模型保存在当前目录下。使用MindSpore Vision中的`DownLoad`下载预训练模型文件到当前目录下，示例代码如下所示：

In [ ]:
from mindvision.dataset import DownLoad

models_url2 = "https://download.mindspore.cn/vision/classification/mobilenet_v2_1.0_224.ckpt"

dl2 = DownLoad()
# 下载预训练模型文件
dl2.download_url(models_url2)

## MobileNet V2、Resnet50模型微调

### 一、MobileNet V2模型微调

本章使用MobileNet V2的预训练模型进行微调，通过删除MobileNet V2预训练模型中最后一个用于分类的1x1的卷积层的参数，使用全新数据集对模型进行重新训练以更新模型参数。

In [ ]:
import mindspore.nn as nn
from mindspore.train import Model
from mindspore import load_checkpoint, load_param_into_net

from mindvision.engine.loss import CrossEntropySmooth
from typing import Any

from mindvision.classification.models.backbones import MobileNetV2
from mindvision.classification.models.classifiers import BaseClassifier
from mindvision.classification.models.neck import GlobalAvgPooling
from mindvision.classification.models.utils import make_divisible
from mindvision.classification.utils.model_urls import model_urls
from mindvision.utils.load_pretrained_model import LoadPretrainedModel

__all__ = ['mobilenet_v2']

def mobilenet_v2(num_classes: int = 1001,
                 alpha: float = 1.0,
                 round_nearest: int = 8,
                 pretrained: bool = False,
                 resize: int = 224,
                 **kwargs: Any) -> MobileNetV2:
    """Mobilenet v2 structure."""
    backbone = MobileNetV2(alpha=alpha, round_nearest=round_nearest, **kwargs)
    neck = GlobalAvgPooling(keep_dims=True)
    inp_channel = make_divisible(1280 * max(1.0, alpha), round_nearest)
    # head = ConvHead(input_channel=inp_channel, num_classes=num_classes)
    head = CosFace(in_features=inp_channel, out_features=num_classes)
    model = BaseClassifier(backbone, neck, head)

    if pretrained:
        # Download the pre-trained checkpoint file from url, and load
        # checkpoint file.
        arch = "mobilenet_v2_" + str(alpha) + "_" + str(resize)
        LoadPretrainedModel(model, model_urls[arch]).run()

    return model


# 创建模型,其中目标分类数为2，图像输入大小为(224,224)
network = mobilenet_v2(num_classes=130, resize=224)

# 定义优化器
# network_opt = nn.Adam(params=network.trainable_params(), learning_rate=0.01, momentum=0.9)
network_opt = nn.Adam(params=network.trainable_params())

# 定义损失函数
network_loss = CrossEntropySmooth(sparse=True, reduction="mean", smooth_factor=0.1, classes_num=130)

# 定义评价指标
metrics = {"Accuracy": nn.Accuracy()}

# 初始化模型
model = Model(network, loss_fn=network_loss, optimizer=network_opt, metrics=metrics)

> 上述WARNING是由于模型微调需要删除预训练模型的最后一个卷积层的参数，所以加载预训练模型会显示`head.classifier`参数未加载，`head.classifier`参数会使用创建模型时的初始化值。

### 二、ResNet50模型微调

同样的，此处采用Resnet50的预训练模型进行微调，也通过删除MobileNet V2预训练模型中最后一个用于分类的1x1的卷积层的参数，使用全新数据集对模型进行重新训练以更新模型参数。

In [ ]:
import mindspore.nn as nn
from mindspore.train import Model
from mindspore import load_checkpoint, load_param_into_net

from mindvision.classification.models import mobilenet_v2, resnet50
from mindvision.engine.loss import CrossEntropySmooth

# 创建模型,其中目标分类数为2，图像输入大小为(224,224)
# network = mobilenet_v2(num_classes=38, resize=224)
network = resnet50(130)

# 模型参数存入到param_dict
# param_dict = load_checkpoint("./mobilenet_v2_1.0_224.ckpt")
param_dict = load_checkpoint("./resnet50_224.ckpt")

# 获取mobilenet_v2网络最后一个卷积层的参数名
filter_list = [x.name for x in network.head.get_parameters()]

# 删除预训练模型的最后一个卷积层
def filter_ckpt_parameter(origin_dict, param_filter):
    for key in list(origin_dict.keys()):
        for name in param_filter:
            if name in key:
                print("Delete parameter from checkpoint: ", key)
                del origin_dict[key]
                break

filter_ckpt_parameter(param_dict, filter_list)

# 加载预训练模型参数作为网络初始化权重
load_param_into_net(network, param_dict)

# 定义优化器
network_opt = nn.Momentum(params=network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 定义损失函数
network_loss = CrossEntropySmooth(sparse=True, reduction="mean", smooth_factor=0.1, classes_num=130)

# 定义评价指标
metrics = {"Accuracy": nn.Accuracy()}

# 初始化模型
model = Model(network, loss_fn=network_loss, optimizer=network_opt, metrics=metrics)

## Resnet101不使用预训练模型全新训练（修改优化器为Adam）

In [ ]:
from mindvision.classification.models import resnet101

network = resnet101(num_classes=130, pretrained=False)

# 定义优化器
network_opt = nn.Adam(params=network.trainable_params())

# 定义损失函数
network_loss = CrossEntropySmooth(sparse=True, reduction="mean", smooth_factor=0.1, classes_num=130)

# 定义评价指标
metrics = {"Accuracy": nn.Accuracy()}

# 初始化模型
model = Model(network, loss_fn=network_loss, optimizer=network_opt, metrics=metrics)

### 模型训练与评估

训练并评估网络，使用MindSpore Vision中的`mindvision.engine.callback.ValAccMonitor`接口打印训练的损失值和评估精度，且在训练完成后，保存评估精度最高的CKPT文件`best.ckpt`在当前目录下。

### 此处由于训练时间过长，之后我们也将用训练好的本地ckpt文件来进行测试。

resnet50:best_resnet50.ckpt

resnet101:best_resnet101.ckpt

mobilenetv2:best_mobilenetv2.skpt

In [ ]:
from mindvision.engine.callback import ValAccMonitor
from mindspore.train.callback import TimeMonitor

num_epochs = 20

# 模型训练与验证，训练完成后保存验证精度最高的ckpt文件（best.ckpt）到当前目录下
model.train(num_epochs,
            dataset_train,
            callbacks=[ValAccMonitor(model, dataset_val, num_epochs), TimeMonitor()])

### 一、ResNet50----模型推理

定义 `visualize_infer` 函数，我们选择用`Australian_Shepherd`狗种类数据集来进行预测，得到模型训练的准确度。

In [4]:
import numpy as np
from PIL import Image
from mindvision.classification.models import resnet50
from mindspore import Tensor
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train import Model

def listdir2(path, list_name):
    i = 0
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            list_name[i] = file
            i += 1

def visualize_infer(path):
    path_list = os.listdir(path)
    print(path_list)
    average_result = 0
    for i in range(len(path_list)):
        path1 = path + "//" + path_list[i]
        image = Image.open(path1).convert("RGB")
        image = image.resize((224, 224))

        # 归一化处理
        mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255])
        std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255])
        image = np.array(image)
        image = (image - mean) / std
        image = image.astype(np.float32)

        # 图像通道由(h, w, c)转换为(c, h, w)
        image = np.transpose(image, (2, 0, 1))

        # 扩展数据维数为(1, c, h, w)
        image = np.expand_dims(image, axis=0)

        # 定义并加载网络
        net = resnet50(130)
        param_dict = load_checkpoint("/home/user2/LiDexin/best_resnet50.ckpt")
        load_param_into_net(net, param_dict)
        model = Model(net)

        # 模型预测
        pre = model.predict(Tensor(image))
        result = np.argmax(pre)
        list_name2 = {}
        train_path = "/home/user2/LiDexin/datasets/train"
        listdir2(train_path, list_name2)
        class_name = list_name2
        if class_name[result] == 'Australian_Shepherd':
            average_result = average_result + 1
    average_result = average_result / len(path_list)
    return average_result

path = "/home/user2/LiDexin/datasets/infer/affenpinscher"
accuracy = visualize_infer(path).__str__()
print('\n\n\n resnet50_accuracy : ' + accuracy)

['n103218.jpg', 'n103220.jpg', 'n103221.jpg', 'n103223.jpg', 'n103225.jpg', 'n103226.jpg', 'n103229.jpg', 'n103230.jpg', 'n103235.jpg', 'n103236.jpg', 'n103238.jpg', 'n103240.jpg', 'n103241.jpg', 'n103242.jpg', 'n103243.jpg', 'n103244.jpg', 'n103245.jpg', 'n103246.jpg', 'n103247.jpg', 'n103248.jpg', 'n103249.jpg', 'n103250.jpg', 'n103251.jpg', 'n103252.jpg', 'n1032522.jpg', 'n103253.jpg', 'n103254.jpg', 'n103255.jpg', 'n103256.jpg', 'n103258.jpg', 'n103259.jpg', 'n103260.jpg', 'n103261.jpg', 'n103263.jpg', 'n103264.jpg', 'n103265.jpg', 'n103266.jpg', 'n103267.jpg', 'n103268.jpg', 'n103271.jpg', 'n103272.jpg', 'n103273.jpg', 'n103274.jpg', 'n103275.jpg', 'n103276.jpg', 'n103277.jpg', 'n103278.jpg', 'n103280.jpg', 'n103281.jpg', 'n103282.jpg', 'n103283.jpg', 'n103284.jpg', 'n103286.jpg', 'n103287.jpg', 'n103288.jpg', 'n103289.jpg', 'n103291.jpg', 'n103292.jpg', 'n103293.jpg', 'n103294.jpg', 'n103296.jpg', 'n103297.jpg', 'n103301.jpg', 'n103302.jpg', 'n103303.jpg', 'n103307.jpg', 'n103309

### 二、MobileNetv2----模型推理

In [5]:
import numpy as np
from PIL import Image
from mindvision.classification.models import mobilenet_v2
from mindspore import Tensor
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train import Model

def listdir2(path, list_name):
    i = 0
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            list_name[i] = file
            i += 1

def visualize_infer(path):
    path_list = os.listdir(path)
    print(path_list)
    average_result = 0
    for i in range(len(path_list)):
        path1 = path + "//" + path_list[i]
        image = Image.open(path1).convert("RGB")
        image = image.resize((224, 224))

        # 归一化处理
        mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255])
        std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255])
        image = np.array(image)
        image = (image - mean) / std
        image = image.astype(np.float32)

        # 图像通道由(h, w, c)转换为(c, h, w)
        image = np.transpose(image, (2, 0, 1))

        # 扩展数据维数为(1, c, h, w)
        image = np.expand_dims(image, axis=0)

        # 定义并加载网络
        net = mobilenet_v2(num_classes=130, resize=224)
        param_dict = load_checkpoint("/home/user2/LiDexin/best_mobilenetv2.ckpt")
        load_param_into_net(net, param_dict)
        model = Model(net)

        # 模型预测
        pre = model.predict(Tensor(image))
        result = np.argmax(pre)
        list_name2 = {}
        train_path = "/home/user2/LiDexin/datasets/train"
        listdir2(train_path, list_name2)
        class_name = list_name2
        if class_name[result] == 'Australian_Shepherd':
            average_result = average_result + 1
    average_result = average_result / len(path_list)
    return average_result

path = "/home/user2/LiDexin/datasets/infer/affenpinscher"
accuracy = visualize_infer(path).__str__()
print('\n\n\n mobilenetv2_accuracy : ' + accuracy)

['n103218.jpg', 'n103220.jpg', 'n103221.jpg', 'n103223.jpg', 'n103225.jpg', 'n103226.jpg', 'n103229.jpg', 'n103230.jpg', 'n103235.jpg', 'n103236.jpg', 'n103238.jpg', 'n103240.jpg', 'n103241.jpg', 'n103242.jpg', 'n103243.jpg', 'n103244.jpg', 'n103245.jpg', 'n103246.jpg', 'n103247.jpg', 'n103248.jpg', 'n103249.jpg', 'n103250.jpg', 'n103251.jpg', 'n103252.jpg', 'n1032522.jpg', 'n103253.jpg', 'n103254.jpg', 'n103255.jpg', 'n103256.jpg', 'n103258.jpg', 'n103259.jpg', 'n103260.jpg', 'n103261.jpg', 'n103263.jpg', 'n103264.jpg', 'n103265.jpg', 'n103266.jpg', 'n103267.jpg', 'n103268.jpg', 'n103271.jpg', 'n103272.jpg', 'n103273.jpg', 'n103274.jpg', 'n103275.jpg', 'n103276.jpg', 'n103277.jpg', 'n103278.jpg', 'n103280.jpg', 'n103281.jpg', 'n103282.jpg', 'n103283.jpg', 'n103284.jpg', 'n103286.jpg', 'n103287.jpg', 'n103288.jpg', 'n103289.jpg', 'n103291.jpg', 'n103292.jpg', 'n103293.jpg', 'n103294.jpg', 'n103296.jpg', 'n103297.jpg', 'n103301.jpg', 'n103302.jpg', 'n103303.jpg', 'n103307.jpg', 'n103309

### 三、ResNet101----模型推理（此网络是尝试全新训练，未加载预训练模型直接通过数据集训练得到的模型）

#### 由于时间问题，且此模型较大，我们只训练了50个epoch,效果达到了如下图的效果：其验证精度值大概在70%

![resnet101结果图.png](./image/resnet101结果图.png)

#### 且由于ResNet101的模型过于复杂，参数量较大，即使是下载了ckpt文件在自己的服务器进行测试还是暴显存，且可能时间有限，其达到的精度也不是很理想，

#### 所以我们后续部署中也就没有采用resnet101的模型。



In [4]:
import numpy as np
from PIL import Image
from mindvision.classification.models import resnet101
from mindspore import Tensor
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train import Model

def listdir2(path, list_name):
    i = 0
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            list_name[i] = file
            i += 1

def visualize_infer(path):
    path_list = os.listdir(path)
    print(path_list)
    average_result = 0
    for i in range(len(path_list)):
        path1 = path + "//" + path_list[i]
        image = Image.open(path1).convert("RGB")
        image = image.resize((224, 224))

        # 归一化处理
        mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255])
        std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255])
        image = np.array(image)
        image = (image - mean) / std
        image = image.astype(np.float32)

        # 图像通道由(h, w, c)转换为(c, h, w)
        image = np.transpose(image, (2, 0, 1))

        # 扩展数据维数为(1, c, h, w)
        image = np.expand_dims(image, axis=0)

        # 定义并加载网络
        net = resnet101(num_classes=130)
        param_dict = load_checkpoint("/home/user2/LiDexin/best_resnet101.ckpt")
        load_param_into_net(net, param_dict)
        model = Model(net)

        # 模型预测
        pre = model.predict(Tensor(image))
        result = np.argmax(pre)
        list_name2 = {}
        train_path = "/home/user2/LiDexin/datasets/train"
        listdir2(train_path, list_name2)
        class_name = list_name2
        if class_name[result] == 'Australian_Shepherd':
            average_result = average_result + 1
    average_result = average_result / len(path_list)
    return average_result

path = "/home/user2/LiDexin/datasets/infer/affenpinscher"
accuracy = visualize_infer(path).__str__()
print('\n\n\n mobilenetv2_accuracy : ' + accuracy)

['n103218.jpg', 'n103220.jpg', 'n103221.jpg', 'n103223.jpg', 'n103225.jpg', 'n103226.jpg', 'n103229.jpg', 'n103230.jpg', 'n103235.jpg', 'n103236.jpg', 'n103238.jpg', 'n103240.jpg', 'n103241.jpg', 'n103242.jpg', 'n103243.jpg', 'n103244.jpg', 'n103245.jpg', 'n103246.jpg', 'n103247.jpg', 'n103248.jpg', 'n103249.jpg', 'n103250.jpg', 'n103251.jpg', 'n103252.jpg', 'n1032522.jpg', 'n103253.jpg', 'n103254.jpg', 'n103255.jpg', 'n103256.jpg', 'n103258.jpg', 'n103259.jpg', 'n103260.jpg', 'n103261.jpg', 'n103263.jpg', 'n103264.jpg', 'n103265.jpg', 'n103266.jpg', 'n103267.jpg', 'n103268.jpg', 'n103271.jpg', 'n103272.jpg', 'n103273.jpg', 'n103274.jpg', 'n103275.jpg', 'n103276.jpg', 'n103277.jpg', 'n103278.jpg', 'n103280.jpg', 'n103281.jpg', 'n103282.jpg', 'n103283.jpg', 'n103284.jpg', 'n103286.jpg', 'n103287.jpg', 'n103288.jpg', 'n103289.jpg', 'n103291.jpg', 'n103292.jpg', 'n103293.jpg', 'n103294.jpg', 'n103296.jpg', 'n103297.jpg', 'n103301.jpg', 'n103302.jpg', 'n103303.jpg', 'n103307.jpg', 'n103309

[WARNING] PRE_ACT(24147,7f343a7fc700,python):2022-07-03-23:24:53.867.981 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:255] CalMemBlockAllocSize] Device memory size [4063232] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(24147,7f343a7fc700,python):2022-07-03-23:24:53.868.009 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:525] DumpDynamicMemPoolDebugInfo] Start dump dynamic memory pool debug info.
[WARNING] PRE_ACT(24147,7f343a7fc700,python):2022-07-03-23:24:53.874.009 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:528] DumpDynamicMemPoolDebugInfo] Finish dump dynamic memory pool debug info.
[WARNING] PRE_ACT(24147,7f343a7fc700,python):2022-07-03-23:24:53.876.326 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:255] CalMemBlockAllocSize] Device memory size [4063232] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(24147,7f343a7fc700,python):2022-07-03-23:24:53.876.340 [mindspore/ccsrc/common/mem_reuse/mem_

[CRITICAL] RUNTIME_FRAMEWORK(24147,7f34f7cc52c0,python):2022-07-03-23:24:55.876.336 [mindspore/ccsrc/runtime/graph_scheduler/graph_scheduler.cc:619] Run] Device(id:0) memory isn't enough and alloc failed, kernel name: Default/backbone-ResNet/conv1-ConvNormActivation/features-SequentialCell/0-Conv2d/Conv2D-op117047, alloc size: 3211264B.


RuntimeError: mindspore/ccsrc/runtime/graph_scheduler/graph_scheduler.cc:619 Run] Device(id:0) memory isn't enough and alloc failed, kernel name: Default/backbone-ResNet/conv1-ConvNormActivation/features-SequentialCell/0-Conv2d/Conv2D-op117047, alloc size: 3211264B.

### 模型导出

在模型训练完后，我们综合三个模型训练时的验证精度和测试集的推理精度综合，选取综合最优的网络模型（即CKPT文件）转换为MindIR格式，用于后续手机侧的推理。通过`export`接口会在当前目录下会生成`model_v2_1.0_224.mindir`文件。

In [9]:
from mindspore import export, Tensor

# 定义并加载网络参数
net = mobilenet_v2(num_classes=130, resize=224)
#param_dict = load_checkpoint("best.ckpt")
#param_dict = load_checkpoint("best.ckpt")
param_dict = load_checkpoint("/home/user2/LiDexin/best_mobilenetv2.ckpt")
load_param_into_net(net, param_dict)

# 将模型由ckpt格式导出为MINDIR格式
input_np = np.random.uniform(0.0, 1.0, size=[1, 3, 224, 224]).astype(np.float32)
export(net, Tensor(input_np), file_name="model_v2_1.0_224", file_format="MINDIR")
print('success')

success


## 手机侧推理与部署

为实现模型文件在手机侧的推理功能，步骤如下：

- 转换文件格式：将MindIR文件格式，转换成Android手机上MindSpore Lite可识别文件；
- 应用部署：在手机侧部署应用APK，即下载一个MindSpore Vision套件Android APK；
- 应用体验：最后将ms模型文件导入到手机侧后，体验狗的分类识别功能。

### 应用体验

将上述训练的自定义网络模型mobilenet_v2_1.0_224.ms部署到Android手机侧，体验狗的分类识别功能。

#### 自定义模型标签文件

自定义模型部署需要按照如下格式定义网络模型需要用到的信息，即自定义标签文件，并在本地电脑端侧创建一个必须以`custom.json`命名的json格式标签文件。

```text
{
    "title": '狗品种分类',
    "file": 'model_v2_1.0_224.ms',
    "label": ['Airedale', 'miniature_poodle', 'affenpinscher', 'schipperke', 'Australian_terrier', 'Welsh_springer_spaniel', 'curly_coated_retriever', 'Staffordshire_bullterrier', 'Norwich_terrier', 'Tibetan_terrier', 'English_setter', 'Norfolk_terrier', 'Pembroke', 'Tibetan_mastiff', 'Border_terrier', 'Great_Dane', 'Scotch_terrier', 'flat_coated_retriever', 'Saluki', 'Irish_setter', 'Blenheim_spaniel', 'Irish_terrier', 'bloodhound', 'redbone', 'West_Highland_white_terrier', 'Brabancon_griffo', 'dhole', 'kelpie', 'Doberman', 'Ibizan_hound', 'vizsla', 'cairn', 'German_shepherd', 'African_hunting_dog', 'Dandie_Dinmont', 'Sealyham_terrier', 'German_short_haired_pointer', 'Bernese_mountain_dog', 'Saint_Bernard', 'Leonberg', 'Bedlington_terrier', 'Newfoundland', 'Lhasa', 'Chesapeake_Bay_retriever', 'Lakeland_terrier', 'Walker_hound', 'American_Staffordshire_terrier', 'otterhound', 'Sussex_spaniel', 'Norwegian_elkhound', 'bluetick', 'dingo', 'Irish_water_spaniel', 'Fila Braziliero', 'standard_schnauzer', 'Mexican_hairless', 'EntleBucher', 'Afghan_hound', 'kuvasz', 'English_foxhound', 'keeshond', 'Irish_wolfhound', 'Scottish_deerhound', 'Rottweiler', 'black_and_tan_coonhound', 'Great_Pyrenees', 'boxer', 'wire_haired_fox_terrier', 'borzoi', 'groenendael', 'collie', 'Gordon_setter', 'Kerry_blue_terrier', 'briard', 'Rhodesian_ridgeback', 'Boston_bull', 'bull_mastiff', 'silky_terrier', 'Brittany_spaniel', 'Eskimo_dog', 'giant_schnauzer', 'malinois', 'Bouvier_des_Flandres', 'whippet', 'Appenzeller', 'Chinese_Crested_Dog', 'soft_coated_wheaten_terrier', 'Weimaraner', 'clumber', 'Greater_Swiss_Mountain_dog', 'toy_terrier', 'Italian_greyhound', 'basset', 'basenji', 'Australian_Shepherd', 'Maltese_dog', 'Japanese_spaniel', 'Cane_Carso', 'Japanese_Spitzes', 'Old_English_sheepdog', 'Black_sable', 'Shetland_sheepdog', 'English_springer', 'beagle', 'cocker_spaniel', 'standard_poodle', 'komondor', 'chow', 'Yorkshire_terrier', 'Shih_Tzu', 'Chihuahua', 'Pekinese', 'miniature_pinscher', 'pug', 'papillon', 'Shiba_Dog', 'French_bulldog', 'Siberian_hus]
}
```

Json标签文件中需包含`title`，`file`，`label`三个Key值字段，其含义如下：

- title ：自定义模块标题(狗品种分类)；
- file ：上文转换好的模型文件名称；
- label ：自定义标签label的`数组`信息。

#### 标签与模型文件部署到手机

在`MindSpore Vision APK`的首页上长按`分类`按钮，可以进入自定义分类模式，并且选择需要部署的标签和模型文件。

为实现手机端狗的分类识别功能，需将标签文件`custom.json`文件和模型文件`model_v2_1.0_224.ms`一起放置到手机上指定目录下。这里以`Android/data/Download/` 文件夹为例，首先把标签文件和模型文件同时放在上述手机地址，如图所示，点击自定义按钮，然后会弹出系统文件功能，点击左上角的打开文件，然后找到Json标签文件和模型文件存放的目录地址，并选择对应的Json文件。

标签与模型文件部署到手机后，即可点击中间按钮进行拍照获取图片，或者点击上侧栏的图像按钮选择图片相册用于图像，就可以进行狗品种分类识别。

## 实际效果

### 但是由于分类的数目过多，导致物体识别的置信度很低，但是准确率还是能达到比较好的效果。

![手机1.png](./image/手机1.jpg)
![手机2.png](./image/手机2.jpg)